In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_excel("./process_data/dining area.xlsx")
df.head()

,Unnamed: 0,start_time,end_time,temperature,luminosity,humidity,motion
0,0,2022-05-01 00:00:00,2022-05-01 01:30:00,22.031250,0.0,46.61,0
1,1,2022-05-01 01:30:00,2022-05-01 03:00:00,21.865000,0.0,48.00,1
2,2,2022-05-01 03:00:00,2022-05-01 04:30:00,21.443333,3.0,48.50,1
3,3,2022-05-01 04:30:00,2022-05-01 06:00:00,21.360000,69.0,49.00,0
4,4,2022-05-01 06:00:00,2022-05-01 07:30:00,20.992500,94.0,48.00,0


In [4]:
df["temperature"].describe()

count    7010.000000
mean       23.561677
std         3.609479
min         4.825109
25%        20.421250
50%        23.367222
75%        26.501500
max        35.401667
Name: temperature, dtype: float64

In [5]:
df["humidity"].describe()

count    7010.000000
mean       52.576881
std        11.712454
min         6.936949
25%        44.000000
50%        51.000000
75%        61.000000
max        81.500000
Name: humidity, dtype: float64